![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [45]:
import numpy as np
import pandas as pd
import xgboost as xgb
from datetime import datetime
from QuantConnect.Research import QuantBook

##

# Get Data

In [8]:
# QuantBook Analysis Tool
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()

qb.set_time_zone(TimeZones.UTC)
start_time = datetime(2015, 5, 1)
end_time = datetime(2025, 5, 17)

symbol = qb.add_crypto("BTCUSDT", Resolution.Minute, Market.BINANCE).Symbol
vix = qb.add_data(CBOE, "VIX", Resolution.Daily).Symbol

In [11]:
hist = qb.History(symbol, start_time, end_time, Resolution.Hour)
vix_hist = qb.History(vix, start_time, end_time, Resolution.Hour)

df = hist.reset_index()
vix_df = vix_hist.reset_index()

df = df[df['symbol'] == symbol.ID.Symbol].copy()
df.set_index('time', inplace=True)
df.head()


vix_df = vix_df[vix_df['symbol'] == vix.ID.Symbol].copy()
vix_df.set_index('time', inplace=True)


In [10]:
vix_df.head()

# Feature Engineering

In [ ]:
df['return_pct1'] = df['close'].pct_change(1)
df['return_pct2'] = df['close'].pct_change(2)
df['return_pct5'] = df['close'].pct_change(5)
df["sma20"] = df['close'].rolling(window=20).mean()
df["sma50"] = df['close'].rolling(window=50).mean()
df["sma200"] = df['close'].rolling(window=200).mean()
df["vol20"] = df['close'].rolling(window=20).std()
df["zscore20"] = ((df['close'] - df['sma20']) / df['vol20'] + 1e-9)

df["target"] = df["return_pct1"].shift(-1)

df.dropna(inplace=True)

df.head()

In [86]:
print(df.shape[0]-124)

In [76]:
df = df.iloc[124:]
df.shape

In [82]:
X = df.drop(columns=["target", "symbol"]).values
y = df["target"].values

In [92]:
def rolling_window_training(X, y, training_size = 180 * 24, test_size = 30 * 24):
    predictions = []
    actuals = []
    for i in range(0, int(len(X) / training_size)):
        X_train = X[i * training_size:(i + 1) * training_size]
        y_train = y[i * training_size:(i + 1) * training_size]
        X_test = X[(i + 1) * training_size:(i + 1) * training_size + test_size]
        y_test = y[(i + 1) * training_size:(i + 1) * training_size + test_size]

        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        predictions.extend(y_pred)
        actuals.extend(y_test)

    return np.array(predictions), np.array(actuals)

In [93]:
predictions, actuals = rolling_window_training(X, y)

In [89]:
print(predictions)
print(actuals)